In [1]:
import re

In [2]:
LINE = re.compile(r'\s*(\w+)\s*-\s*(\w+)\s*=\s*(\d+)')

In [3]:
def parse_line(line):
    m = LINE.match(line)
    if m:
        node1 = m.group(1)
        node2 = m.group(2)
        weight = int(m.group(3))
        
        return node1, node2, weight
    return None

In [4]:
def parse_lines(lines):
    relations = []
    for i, line in enumerate(lines):
        p = parse_line(line)
        if p is None:
            raise ValueError('Invalid syntax on line {}: "{}"'.format(i + 1, line))
        relations.append(p)
    return relations

In [5]:
def node_names(relations):
    nodes = set()
    idx = 0
    for i in relations:
        nodes.add(i[0])
        nodes.add(i[1])
    return nodes

In [6]:
def node_index_table(nodes):
    tab = {}
    idx = 0
    
    for i in nodes:
        if i not in tab.keys():
            tab[i] = idx
            idx += 1
    
    return tab

In [39]:
class Graph(object):
    def __init__(self, filename):
        with open(filename, 'r') as f:
            l = f.readlines()
        
        self.relations = parse_lines(l)
        self.nodes = node_names(self.relations)
        self.index = node_index_table(self.nodes)
        self._matrix()
        
    def _matrix(self):
        self.mat = [[-1 for _ in self.nodes] for _ in self.nodes]
        for a, b, w in self.relations:
            self._set_weight(a, b, w)
            self._set_weight(b, a, w)
            self._set_weight(a, a, 0)
            self._set_weight
        
    def get_weight(self, x, y):
        x_idx = self.index[x]
        y_idx = self.index[y]
        
        return self.mat[x_idx][y_idx]
    
    def _set_weight(self, x, y, w):
        x_idx = self.index[x]
        y_idx = self.index[y]
        
        self.mat[x_idx][y_idx] = w
        
    def dijkstra(self, n):
        v = {n}
        path = {n: (0,n)}
        
        while v != self.nodes:
            node, weight, pre = self.nearest_neighbor(v)
            v.add(node)
            weight += path[pre][0]
            path[node] = (weight, pre)
        return path
    
    def prims(self):
        y = self.nodes.copy()
        x = {y.pop()}
        t = []
        
        while x != self.nodes:
            u, v, w = self.cheapest_cut_edge(x, y)
            x.add(v)
            y.remove(v)
            t.append((u, v, w))
            
        return t

    def print_prims(self):
        p = self.prims()
        
        for pre, node, weight in p:
            print('{}->{} {}'.format(pre, node, weight))
            
    def cheapest_cut_edge(self, X, Y):
        v, w, u = self.nearest_neighbor(X)
        return u, v, w
    
    def print_dijkstra(self, n):
        d = self.dijkstra(n)
        l = [n]
        
        while len(l) < len(self.nodes):
            x = l[-1]
            for node, (weight, pre) in d.items():
                if pre == x and node not in l:
                    arrows = '->'.join(l[::-1])
                    print('{}->{} {}'.format(arrows, node, weight))
                    l.append(node)

    def nearest_neighbor(self,v):
        return min(self.neighbors(v), key = lambda x:x[1])
        
    def neighbors(self, v):
        neighbors = []
        for v_node in v:
            for self_node in self.nodes:
                weight = self.get_weight(v_node, self_node)
                if weight != 0 and weight != -1 and self_node not in v:
                    neighbors.append((self_node, weight, v_node))
        return neighbors

In [8]:
m = Matrix(['a-b=5', 'a-c=10'])
print(m.mat)
print(m.index)

[[-1, 10, -1], [10, 0, 5], [-1, 5, -1]]
{'a': 1, 'c': 0, 'b': 2}


In [9]:
def mat_for_file(filename):
    with open(filename, 'r') as f:
        l = f.readlines()
        return Matrix(l)

In [40]:
m = Graph('matrix1')
m.print_prims()

c->e 4
e->b 2
b->a 1
a->d 4
c->f 5
